In [1]:
%load_ext autoreload
%autoreload 2

import polars as pl
from pre_snap_prediction.data import process_data, process_orpsp

In [2]:
player_play = pl.read_csv("../data/player_play.csv", null_values="NA")

In [3]:
players = pl.read_csv("../data/players.csv", null_values="NA")

In [4]:
plays = pl.read_csv("../data/plays.csv", null_values="NA")

In [5]:
plays["offenseFormation"].value_counts()

offenseFormation,count
str,u32
"""SINGLEBACK""",3915
"""SHOTGUN""",8791
"""I_FORM""",1035
"""PISTOL""",641
"""JUMBO""",125
"""EMPTY""",1342
"""WILDCAT""",87
null,188


In [6]:
plays.filter(pl.col("offenseFormation")=="SHOTGUN")["receiverAlignment"].value_counts()

receiverAlignment,count
str,u32
"""2x1""",428
"""3x1""",4085
"""1x1""",7
"""1x0""",1
"""3x2""",1
"""2x2""",4268
"""3x0""",1


In [7]:
plays.filter(pl.col("offenseFormation")=="SHOTGUN", pl.col("receiverAlignment")=="2x2")

gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,playNullifiedByPenalty,absoluteYardlineNumber,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,expectedPoints,offenseFormation,receiverAlignment,playClockAtSnap,passResult,passLength,targetX,targetY,playAction,dropbackType,dropbackDistance,passLocationType,timeToThrow,timeInTackleBox,timeToSack,passTippedAtLine,unblockedPressure,qbSpike,qbKneel,qbSneak,rushLocationType,penaltyYards,prePenaltyYardsGained,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
i64,i64,str,i64,i64,i64,str,str,str,i64,str,i64,i64,str,i64,f64,f64,f64,str,str,i64,str,i64,f64,f64,bool,str,f64,str,f64,f64,f64,bool,bool,bool,i64,bool,str,i64,i64,i64,f64,f64,f64,bool,str,str,i64,str,str
2022103004,3146,"""(2:00) (Shotgun) D.Mills pass …",4,3,12,"""HOU""","""TEN""","""HOU""",20,"""02:00""",3,17,"""N""",30,0.006291,0.993709,-0.291485,"""SHOTGUN""","""2x2""",12,"""C""",-4,26.02,17.56,false,"""TRADITIONAL""",3.2,"""INSIDE_BOX""",2.236,2.236,null,false,false,false,0,null,null,null,6,6,0.000205,-0.000205,-0.21848,true,null,null,0,"""Quarters""","""Zone"""
2022110610,348,"""(9:28) (Shotgun) P.Mahomes pas…",1,2,10,"""KC""","""TEN""","""TEN""",23,"""09:28""",0,0,"""N""",33,0.884223,0.115777,4.249382,"""SHOTGUN""","""2x2""",11,"""C""",-6,38.95,14.19,false,"""TRADITIONAL""",3.02,"""INSIDE_BOX""",2.202,2.202,null,false,false,false,0,null,null,null,4,4,-0.001308,0.001308,-0.427749,true,null,null,0,"""Quarters""","""Zone"""
2022110605,3861,"""(:29) (Shotgun) J.Wilkins up t…",4,1,10,"""IND""","""NE""","""IND""",40,"""00:29""",26,3,"""N""",50,0.997811,0.002189,0.991169,"""SHOTGUN""","""2x2""",18,"""""",null,null,null,false,null,null,null,null,null,null,null,null,null,0,false,"""OUTSIDE_LEFT""",null,5,5,0.002189,-0.002189,-0.991169,false,"""INSIDE ZONE""",null,0,"""Cover-2""","""Zone"""
2022100912,445,"""(8:51) (Shotgun) C.Rush pass i…",1,3,5,"""DAL""","""LA""","""LA""",15,"""08:51""",0,6,"""N""",95,0.411201,0.588799,3.963614,"""SHOTGUN""","""2x2""",16,"""I""",5,100.04,3.51,false,"""TRADITIONAL""",2.27,"""INSIDE_BOX""",2.13,2.13,null,false,false,false,0,null,null,null,0,0,0.034006,-0.034006,-1.288449,true,null,null,0,"""Cover-3 Seam""","""Zone"""
2022110603,2325,"""(7:36) (No Huddle, Shotgun) A.…",3,2,9,"""GB""","""DET""","""GB""",47,"""07:36""",8,0,"""N""",63,0.657711,0.342289,2.303506,"""SHOTGUN""","""2x2""",7,"""C""",12,51.28,2.65,false,"""TRADITIONAL""",4.76,"""INSIDE_BOX""",2.97,2.97,null,false,false,false,0,null,null,null,12,12,-0.044392,0.044392,1.286698,true,null,null,0,"""Quarters""","""Zone"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022102303,1153,"""(8:50) (Shotgun) J.Goff pass s…",2,3,14,"""DET""","""DAL""","""DAL""",35,"""08:50""",3,3,"""N""",45,0.716269,0.283731,2.217553,"""SHOTGUN""","""2x2""",7,"""C""",-4,48.84,32.08,false,"""TRADITIONAL""",4.61,"""INSIDE_BOX""",1.935,1.935,null,false,true,false,0,null,null,null,1,1,0.004773,-0.004773,-0.827386,true,null,null,0,"""Cover-3""","""Zone"""
2022110600,1138,"""(8:12) (Shotgun) J.Herbert pas…",2,2,15,"""LAC""","""ATL""","""ATL""",41,"""08:12""",10,0,"""N""",51,0.727737,0.272263,2.273283,"""SHOTGUN""","""2x2""",14,"""C""",4,46.24,35.7,false,"""TRADITIONAL""",4.05,"""INSIDE_BOX""",2.97,2.97,null,false,false,false,0,null,null,null,6,6,-0.009573,0.009573,0.388327,true,null,null,0,"""Cover-2""","""Zone"""
2022110604,1051,"""(12:49) (Shotgun) T.Lawrence p…",2,3,4,"""JAX""","""LV""","""JAX""",31,"""12:49""",0,10,"""N""",79,0.207444,0.792556,0.702276,"""SHOTGUN""","""2x2""",9,"""I""",6,72.69,39.92,false,"""TRADITIONAL""",1.67,"""INSIDE_BOX""",1.86,1.86,null,false,false,false,0,null,null,null,0,0,-0.024741,0.024741,-1.391687,true,null,null,0,"""Cover-2""",""

In [8]:
complete_plays = pl.read_csv("../data/complete_plays.csv", null_values=["NA", ""])

In [9]:
tracking = process_data.read_tracking_csv()

In [10]:
tracking_features = process_orpsp.get_tracking_features(complete_plays, tracking)

In [11]:
start_features = process_orpsp.get_start_features(tracking_features)

In [12]:
start_features

gameId,playId,nflId,frameType,frameId,playDirection,x,y,o,event,route_left,x_ball,y_ball
i64,i64,i64,str,i64,str,f64,f64,f64,str,bool,f64,f64
2022103003,1955,53959,"""BEFORE_SNAP""",95,"""left""",33.96,23.58,73.28,null,true,35.25,27.939999
2022091804,280,46132,"""BEFORE_SNAP""",118,"""left""",98.79,13.28,309.24,null,false,92.68,23.46
2022092511,3115,53611,"""BEFORE_SNAP""",87,"""right""",50.37,24.2,null,null,true,49.419998,30.1
2022091100,2533,48520,"""BEFORE_SNAP""",100,"""left""",54.91,23.0,279.06,null,true,54.07,23.53
2022091805,2066,52446,"""BEFORE_SNAP""",77,"""left""",35.54,34.02,253.8,null,false,34.970001,29.79
…,…,…,…,…,…,…,…,…,…,…,…,…
2022102301,1937,53499,"""BEFORE_SNAP""",121,"""left""",28.78,25.18,123.2,null,false,30.25,23.179999
2022110603,1288,47827,"""BEFORE_SNAP""",99,"""left""",27.96,31.23,79.72,null,false,28.889999,29.359999
2022102304,1610,44830,"""BEFORE_SNAP""",103,"""left""",52.54,15.19,290.05,null,false,44.230003,23.330001


In [13]:
plays.filter(pl.col("offenseFormation")=="SHOTGUN", pl.col("receiverAlignment")=="2x2").sort("gameId")

gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,playNullifiedByPenalty,absoluteYardlineNumber,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,expectedPoints,offenseFormation,receiverAlignment,playClockAtSnap,passResult,passLength,targetX,targetY,playAction,dropbackType,dropbackDistance,passLocationType,timeToThrow,timeInTackleBox,timeToSack,passTippedAtLine,unblockedPressure,qbSpike,qbKneel,qbSneak,rushLocationType,penaltyYards,prePenaltyYardsGained,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
i64,i64,str,i64,i64,i64,str,str,str,i64,str,i64,i64,str,i64,f64,f64,f64,str,str,i64,str,i64,f64,f64,bool,str,f64,str,f64,f64,f64,bool,bool,bool,i64,bool,str,i64,i64,i64,f64,f64,f64,bool,str,str,i64,str,str
2022090800,1657,"""(1:48) (Shotgun) M.Stafford pa…",2,2,10,"""LA""","""BUF""","""BUF""",45,"""01:48""",7,10,"""N""",55,0.40665,0.59335,2.465821,"""SHOTGUN""","""2x2""",6,"""I""",23,40.37,32.14,false,"""TRADITIONAL""",3.14,"""INSIDE_BOX""",2.436,2.436,null,false,false,false,0,null,null,null,0,0,-0.014103,0.014103,-0.479244,true,null,null,0,"""Cover-3""","""Zone"""
2022090800,2464,"""(5:11) (Shotgun) J.Allen scram…",3,1,15,"""BUF""","""LA""","""BUF""",6,"""05:11""",10,17,"""N""",104,0.219183,0.780817,-0.670623,"""SHOTGUN""","""2x2""",9,"""R""",null,null,null,false,"""SCRAMBLE""",1.92,null,null,null,null,null,false,null,0,false,"""INSIDE_LEFT""",null,8,8,-0.019701,0.019701,0.578808,true,null,null,0,"""Cover-3""","""Zone"""
2022090800,2688,"""(15:00) (Shotgun) J.Allen pass…",4,3,7,"""BUF""","""LA""","""BUF""",47,"""15:00""",10,17,"""N""",57,0.150378,0.849622,1.207212,"""SHOTGUN""","""2x2""",19,"""C""",40,95.74,31.79,false,"""TRADITIONAL""",3.12,"""INSIDE_BOX""",2.836,2.836,null,false,false,false,0,null,null,null,47,47,-0.067931,0.067931,4.467861,true,null,null,0,"""Cover-3""","""Zone"""
2022090800,467,"""(6:21) (No Huddle, Shotgun) M.…",1,2,6,"""LA""","""BUF""","""BUF""",43,"""06:21""",0,7,"""N""",67,0.324274,0.675726,2.919861,"""SHOTGUN""","""2x2""",20,"""S""",null,null,null,false,"""TRADITIONAL""",3.69,"""INSIDE_BOX""",null,null,5.205,false,false,false,0,null,null,null,-8,-8,-0.052261,0.052261,-2.094141,true,null,null,0,"""Cover-3""","""Zone"""
2022090800,2572,"""(1:45) (Shotgun) J.Allen pass …",3,2,9,"""BUF""","""LA""","""BUF""",34,"""01:45""",10,17,"""N""",76,0.163963,0.836037,1.050046,"""SHOTGUN""","""2x2""",8,"""C""",2,73.96,20.11,false,"""TRADITIONAL""",2.44,"""INSIDE_BOX""",2.269,2.269,null,false,false,false,0,null,null,null,4,4,0.006451,-0.006451,-0.29679,true,null,null,0,"""Cover-3 Seam""","""Zone"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2022110700,1692,"""(1:56) (Shotgun) A.Kamara righ…",2,2,10,"""NO""","""BAL""","""NO""",25,"""01:56""",0,14,"""N""",35,0.162649,0.837351,0.932146,"""SHOTGUN""","""2x2""",13,"""""",null,null,null,false,null,null,null,null,null,null,null,null,null,0,false,"""INSIDE_RIGHT""",null,9,9,0.009386,-0.009386,0.537077,false,"""DRAW""",null,0,"""Quarters""","""Zone"""
2022110700,2465,"""(5:43) 14-A.Dalton returns at …",3,2,4,"""NO""","""BAL""","""BAL""",39,"""05:43""",3,17,"""N""",49,0.158891,0.841109,3.396368,"""SHOTGUN""","""2x2""",3,"""S""",null,null,null,false,"""TRADITIONAL""",2.58,"""INSIDE_BOX""",null,null,4.458,false,false,false,0,null,null,null,-2,-2,-0.017459,0.017459,-0.90608,true,null,null,0,"""Cover-3""","""Zone"""
2022110700,3581,"""(3:29) (Shotgun) L.Jackson lef…",4,2,4,"""BAL""","""NO""","""BAL""",28,"""03:29""",13,27,"""N""",82,0.00515,0.99485,0.939215,"""SHOTGUN""","""2x2""",5,"""""",null,null,null,true,"""DESIGNED_RUN""",5.02,null,null,null,null,null,null,null,0,false,"""INSIDE_LEFT""",null,7,7,-0.001013,0.001013,0.783922,false,"""POWER""","""INVERTED;READ OPTIO

In [14]:
test = start_features.filter(
    pl.col("gameId")==2022090800, 
    pl.col("playId")==467,
).join(
    player_play.select(["gameId", "playId", "nflId", "teamAbbr", "wasRunningRoute"]),
    on=["gameId", "playId", "nflId"],
    how="inner",
).join(
    plays.select(["gameId", "playId", "possessionTeam"]),
    on=["gameId", "playId"],
    how="inner",
).join(
    players.select(["nflId", "position"]),
    on=["nflId"],
    how="inner",
)

In [15]:
test = test.filter(pl.col("teamAbbr")==pl.col("possessionTeam"))

In [16]:
shotgun_2x2 = test.with_columns(
    (pl.col("x")-pl.col("x_ball")).alias("x"),
    (pl.col("y")-pl.col("y_ball")+53.3/2).alias("y"),
    pl.col("wasRunningRoute").fill_null(0).alias("wasRunningRoute")
).select(["x", "y", "wasRunningRoute", "position"]).sort("y", descending=True)

In [17]:
default_formations = {
    "SHOTGUN": {
        "2x2": shotgun_2x2.to_dict(as_series=False)
    }
}

In [18]:
default_formations

{'SHOTGUN': {'2x2': {'x': [-1.6899999999999977,
    -0.4200000000000017,
    -0.9300000000000068,
    -4.670000000000002,
    -0.7800000000000011,
    -4.68,
    -0.12000000000000455,
    -0.7099999999999937,
    -0.8499999999999943,
    -1.3599999999999994,
    -0.5100000000000051],
   'y': [37.33,
    35.79,
    29.67,
    28.66,
    28.16,
    26.799999999999997,
    26.59,
    25.119999999999997,
    23.72,
    20.5,
    14.369999999999997],
   'wasRunningRoute': [1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1],
   'position': ['WR', 'WR', 'T', 'RB', 'G', 'QB', 'C', 'C', 'T', 'TE', 'WR']}}}

In [19]:
import json
with open("../src/pre_snap_prediction/plays_designer/default_formations.json", "w") as outfile: 
    json.dump(default_formations, outfile)